In [1]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, StandardScaler, RobustScaler
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from missingpy import KNNImputer

import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
path = '../home_credit_data'
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))
from memory_profiler import profile

from IPython.display import display, HTML

# from imputer import Imputer
from sklearn.preprocessing import Imputer
from missingpy import KNNImputer
import lightgbm as lgb
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, StandardScaler, RobustScaler
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier

## 0. Retrieving Data

In [2]:
df = pd.read_csv(path+'/home_credit_processed.csv')

In [3]:
# checking missing data
total = df.isnull().sum().sort_values(ascending = False)
percent = (df.isnull().sum()/df.isnull().count()*100).sort_values(ascending = False)
missing_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
REFUSED_RATE_DOWN_PAYMENT_MEAN,303648,85.234287
REFUSED_AMT_DOWN_PAYMENT_MIN,303648,85.234287
REFUSED_RATE_DOWN_PAYMENT_MAX,303648,85.234287
REFUSED_RATE_DOWN_PAYMENT_MIN,303648,85.234287
REFUSED_AMT_DOWN_PAYMENT_MEAN,303648,85.234287
REFUSED_AMT_DOWN_PAYMENT_MAX,303648,85.234287
REFUSED_APP_CREDIT_PERC_VAR,298034,83.658432
CC_AMT_PAYMENT_CURRENT_VAR,284649,79.901249
CC_CNT_DRAWINGS_ATM_CURRENT_VAR,284559,79.875986
CC_AMT_DRAWINGS_ATM_CURRENT_VAR,284559,79.875986


## 1. Light GBM feature importance 

In [4]:
# Display/plot feature importance
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances092319.png')

### k-fold LightGBM

In [ ]:
debug = False
stratified = False
num_folds = 5
submission_file_name = "results092319.csv"

In [ ]:
# Divide in training/validation and test data
train_df = df[df['TARGET'].notnull()]
test_df = df[df['TARGET'].isnull()]
print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
del df
gc.collect()
# Cross validation model
if stratified:
    folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=1001)
else:
    folds = KFold(n_splits= num_folds, shuffle=True, random_state=1001)
# Create arrays and dataframes to store results
oof_preds = np.zeros(train_df.shape[0])
sub_preds = np.zeros(test_df.shape[0])
feature_importance_df = pd.DataFrame()
feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]

for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
    train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
    valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]

    # LightGBM parameters found by Bayesian optimization
# =============================================================================
#    clf = LGBMClassifier(
#         nthread=4,
#         n_estimators=10000,
#         learning_rate=0.02,
#         num_leaves=34,
#         colsample_bytree=0.9497036,
#         subsample=0.8715623,
#         max_depth=8,
#         reg_alpha=0.041545473,
#         reg_lambda=0.0735294,
#         min_split_gain=0.0222415,
#         min_child_weight=39.3259775,
#         silent=-1,
#         verbose=-1, )
# =============================================================================
    clf = LGBMClassifier(
        nthread=4,
        n_estimators=10000,
        learning_rate=0.02,
        num_leaves=56,
        colsample_bytree=0.45,
        subsample=0.91,
        max_depth=5,
        reg_alpha=3.99,
        reg_lambda=0.43,
        min_split_gain=0.077649,
        min_child_weight=32.5443,
        silent=-1,
        verbose=-1, )

clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
    eval_metric= 'auc', verbose= 100, early_stopping_rounds= 200)

oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
sub_preds += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits

fold_importance_df = pd.DataFrame()
fold_importance_df["feature"] = feats
fold_importance_df["importance"] = clf.feature_importances_
fold_importance_df["fold"] = n_fold + 1
feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))
del clf, train_x, train_y, valid_x, valid_y
gc.collect()

print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))
# Write submission file and plot feature importance
if not debug:
    test_df['TARGET'] = sub_preds
    test_df[['SK_ID_CURR', 'TARGET']].to_csv(submission_file_name, index= False)
display_importances(feature_importance_df)

Starting LightGBM. Train shape: (307507, 799), test shape: (48744, 799)
Training until validation scores don't improve for 200 rounds.
[100]	training's auc: 0.775477	training's binary_logloss: 0.248214	valid_1's auc: 0.763649	valid_1's binary_logloss: 0.250773
[200]	training's auc: 0.789871	training's binary_logloss: 0.239236	valid_1's auc: 0.774013	valid_1's binary_logloss: 0.243577
[300]	training's auc: 0.80018	training's binary_logloss: 0.234562	valid_1's auc: 0.780388	valid_1's binary_logloss: 0.240568
[400]	training's auc: 0.807789	training's binary_logloss: 0.231285	valid_1's auc: 0.783952	valid_1's binary_logloss: 0.238931
[500]	training's auc: 0.813674	training's binary_logloss: 0.228774	valid_1's auc: 0.786395	valid_1's binary_logloss: 0.23787
[600]	training's auc: 0.818572	training's binary_logloss: 0.226662	valid_1's auc: 0.788084	valid_1's binary_logloss: 0.237116
[700]	training's auc: 0.822896	training's binary_logloss: 0.224846	valid_1's auc: 0.789315	valid_1's binary_log